## prune to vpu09

In [ ]:
import os,sys,git,tempfile,psycopg2;
from ipywidgets import IntProgress,HTML,VBox;
from IPython.display import display;
from contextlib import closing;

sys.path.append(os.path.join(os.path.expanduser('~'),'notebooks'));
import common;

dbse = os.environ['POSTGRESQL_DB'];
host = os.environ['POSTGRESQL_HOST'];
port = os.environ['POSTGRESQL_PORT'];
user = 'cipsrv';
pasw = os.environ['POSTGRESQL_CIP_PASS'];

In [ ]:
cs = "dbname=%s user=%s password=%s host=%s port=%s" % (
     dbse
    ,user
    ,pasw
    ,host
    ,port
);

try:
    conn = psycopg2.connect(cs);
except:
    raise Exception("database connection error");

print("database is ready");

In [ ]:
with closing(conn.cursor()) as cursor:
    
    # MR
    cursor.execute("DELETE FROM cipsrv_nhdplus_m.catchment_fabric   a WHERE SUBSTR(a.vpuid,1,2)     != '09'");
    conn.commit();   
    cursor.execute("""DELETE FROM cipsrv_nhdplus_m.catchment_fabric_huc12 a WHERE SUBSTR(a.xwalk_huc12,1,2) != '09' AND a.xwalk_huc12 NOT IN (
        SELECT b.xwalk_huc12 FROM cipsrv_nhdplus_m.catchment_fabric b
    )""");
    cursor.execute("DELETE FROM cipsrv_nhdplus_m.nhdflowline        a WHERE SUBSTR(a.vpuid,1,2)     != '09'");
    cursor.execute("DELETE FROM cipsrv_nhdplus_m.nhdplusflowlinevaa a WHERE SUBSTR(a.vpuid,1,2)     != '09'");
    cursor.execute("DELETE FROM cipsrv_nhdplus_m.nhdplusflow        a WHERE SUBSTR(a.fromvpuid,1,2) != '09'");
    conn.commit();
    
    # HR
    cursor.execute("DELETE FROM cipsrv_nhdplus_h.catchment_fabric   a WHERE SUBSTR(a.vpuid,1,2)     != '09'");
    conn.commit();   
    cursor.execute("""DELETE FROM cipsrv_nhdplus_h.catchment_fabric_huc12 a WHERE SUBSTR(a.xwalk_huc12,1,2) != '09' AND a.xwalk_huc12 NOT IN (
        SELECT b.xwalk_huc12 FROM cipsrv_nhdplus_h.catchment_fabric b
    )""");
    cursor.execute("DELETE FROM cipsrv_nhdplus_h.nhdflowline        a WHERE SUBSTR(a.vpuid,1,2)     != '09'");
    cursor.execute("DELETE FROM cipsrv_nhdplus_h.nhdplusflowlinevaa a WHERE SUBSTR(a.vpuid,1,2)     != '09'");
    cursor.execute("DELETE FROM cipsrv_nhdplus_h.nhdplusflow        a WHERE SUBSTR(a.fromvpuid,1,2) != '09'");
    conn.commit();

    # Support
    cursor.execute("""
       DELETE FROM cipsrv_support.tiger_aiannha_5070 a 
       WHERE NOT ST_INTERSECTS(
           a.shape
          ,(SELECT ST_COLLECT(b.shape) FROM cipsrv_support.tiger_fedstatewaters_5070 b WHERE b.stusps IN ('MN','ND','SD'))
       )""");
    conn.commit();
    cursor.execute("""
        DELETE FROM cipsrv_support.tiger_aiannha a 
        WHERE NOT EXISTS (
            SELECT 1 FROM cipsrv_support.tiger_aiannha_5070 b WHERE b.geoid = a.geoid
        )""");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_aiannha_3338");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_aiannha_26904");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_aiannha_32161");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_aiannha_32655");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_aiannha_32702");
    
    cursor.execute("DELETE FROM cipsrv_support.tiger_fedstatewaters a WHERE a.stusps NOT IN ('MN','ND','SD')");
    cursor.execute("DELETE FROM cipsrv_support.tiger_fedstatewaters_5070 a WHERE a.stusps NOT IN ('MN','ND','SD')");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_fedstatewaters_3338");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_fedstatewaters_26904");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_fedstatewaters_32161");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_fedstatewaters_32655");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_fedstatewaters_32702");
    
    cursor.execute("DELETE FROM cipsrv_support.wbd_hu12_np21 a WHERE SUBSTR(a.huc12,1,2) != '09'");
    conn.commit();


In [ ]:
conn.close();
td.cleanup();
